In [12]:
import torch
import torch.nn.functional as F
import torch.nn as nn
from PIL import Image
from torch import optim
from torchvision import transforms
from torchvision.models import mobilenet_v3_small
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms


In [30]:
# Define image preprocessing transformations
preprocess = transforms.Compose([
    transforms.Resize([224, 224]),  # Resize the image to 224x224
    transforms.ToTensor(),   # Convert the image to a tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the image
])


In [31]:
dataset = datasets.ImageFolder('D:/Download/fas_face_dataset/', transform=preprocess)

total_size = len(dataset)
train_size = int(0.7 * total_size)
val_size = total_size - train_size

# Split the dataset into training and validation sets
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create DataLoader for training and validation sets
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

In [32]:
# Load the pretrained MobileNetV3 model
model = mobilenet_v3_small(pretrained=False)

num_features = model.classifier[3].in_features
model.classifier[3] = nn.Linear(num_features, 2)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [33]:
for name, param in model.named_parameters():
    print(f"Layer: {name}, Size: {param.size()}")

# Print the input size of the model
input_size = (3, 224, 224)  # Input size for MobileNetV3
print(f"Input Size: {input_size}")

Layer: features.0.0.weight, Size: torch.Size([16, 3, 3, 3])
Layer: features.0.1.weight, Size: torch.Size([16])
Layer: features.0.1.bias, Size: torch.Size([16])
Layer: features.1.block.0.0.weight, Size: torch.Size([16, 1, 3, 3])
Layer: features.1.block.0.1.weight, Size: torch.Size([16])
Layer: features.1.block.0.1.bias, Size: torch.Size([16])
Layer: features.1.block.1.fc1.weight, Size: torch.Size([8, 16, 1, 1])
Layer: features.1.block.1.fc1.bias, Size: torch.Size([8])
Layer: features.1.block.1.fc2.weight, Size: torch.Size([16, 8, 1, 1])
Layer: features.1.block.1.fc2.bias, Size: torch.Size([16])
Layer: features.1.block.2.0.weight, Size: torch.Size([16, 16, 1, 1])
Layer: features.1.block.2.1.weight, Size: torch.Size([16])
Layer: features.1.block.2.1.bias, Size: torch.Size([16])
Layer: features.2.block.0.0.weight, Size: torch.Size([72, 16, 1, 1])
Layer: features.2.block.0.1.weight, Size: torch.Size([72])
Layer: features.2.block.0.1.bias, Size: torch.Size([72])
Layer: features.2.block.1.0.w

In [37]:
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    for image,label in train_loader:
        optimizer.zero_grad()
        outputs = model(image)
        loss = criterion(outputs, label)
        loss.backward()
        optimizer.step()
        if epoch % 10 == 0:
            print('Epoch {}/{}, Loss: {:.4f}'.format(epoch + 1, num_epochs, loss.item()))
            #print('Accuracy of the network on the {} test images: {:.2f}'.format(len(val_loader.dataset)))
            
    model.eval()
    total_correct = 0
    total_samples = 0

    with torch.no_grad():
        for image, label in val_loader:
            outputs = model(image)
            _, predicted = torch.max(outputs, 1)
            total_samples += label.size(0)
            total_correct += (predicted == label).sum().item()
            accuracy = total_correct/total_samples
            print('Accuracy of the network on the {} test images: {:.2f}'.format(len(val_loader.dataset), accuracy))
            print('Epoch {}/{}, Loss: {:.4f}'.format(epoch + 1, num_epochs, loss.item()))
            #print('Accuracy of the network on the {} test images: {:.2f}'.format())



Epoch 1/10, Loss: 0.0296
Epoch 1/10, Loss: 0.0028
Epoch 1/10, Loss: 0.0193
Epoch 1/10, Loss: 0.0260
Epoch 1/10, Loss: 0.0145
Epoch 1/10, Loss: 0.0104
Epoch 1/10, Loss: 0.0133
Epoch 1/10, Loss: 0.0011
Epoch 1/10, Loss: 0.0073
Epoch 1/10, Loss: 0.0124
Epoch 1/10, Loss: 0.0026
Epoch 1/10, Loss: 0.0050
Accuracy of the network on the 312 test images: 0.69
Epoch 1/10, Loss: 0.0050
Accuracy of the network on the 312 test images: 0.68
Epoch 1/10, Loss: 0.0050
Accuracy of the network on the 312 test images: 0.64
Epoch 1/10, Loss: 0.0050
Accuracy of the network on the 312 test images: 0.61
Epoch 1/10, Loss: 0.0050
Accuracy of the network on the 312 test images: 0.59
Epoch 1/10, Loss: 0.0050
Accuracy of the network on the 312 test images: 0.69
Epoch 2/10, Loss: 0.0000
Accuracy of the network on the 312 test images: 0.68
Epoch 2/10, Loss: 0.0000
Accuracy of the network on the 312 test images: 0.64
Epoch 2/10, Loss: 0.0000
Accuracy of the network on the 312 test images: 0.61
Epoch 2/10, Loss: 0.000

In [ ]:
torch.save(model.state_dict(), 'mobilenet_v3_binary_classification.pth')